In [13]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
import whisper

model = whisper.load_model("base")


In [15]:
model.device

device(type='cuda', index=0)

In [16]:
from IPython.display import Audio
Audio("/content/Jack Sparrow English Dialogue.mp3")

Detected language: hi
پشپا نام سنکہ فلاورظامجے کیا فائر امے


     |████████████████████████████████| 5.3 MB 8.5 MB/s 
     |████████████████████████████████| 212 kB 69.4 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 112 kB 53.5 MB/s 
     |████████████████████████████████| 270 kB 53.8 MB/s 
     |████████████████████████████████| 2.3 MB 37.9 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
     |████████████████████████████████| 68 kB 7.1 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 856 kB 53.5 MB/s 
     |████████████████████████████████| 4.0 MB 60.6 MB/s 
     |████████████████████████████████| 594 kB 73.5 MB/s 


In [17]:
def transcribe(audio):

    #time.sleep(3)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text


In [18]:
transcribe("/content/Jack Sparrow English Dialogue.mp3")

Detected language: en


"You had the chalice, the mortar, the tear. You could have lived, maybe forever. The fountain does test you, Gibbs, but whether to not know which moment maybe you lost, every morsel of your entire being alive to the infinite mystery at all. I news to sail one bit for an eye. Discoverer of the fountain of youth. I have no say in it, Gibbs. It's a pirate's life for me."

In [8]:

!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00


#TEXT SUMMARIZATION

In [20]:
from transformers import pipeline

In [21]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [19]:
ARTICLE = transcribe("/content/Jack Sparrow English Dialogue.mp3")


Detected language: en


In [22]:
summarizer(ARTICLE, max_length=50, min_length=30, do_sample=False)

[{'summary_text': ' The fountain does test you, Gibbs, but whether to not know which moment maybe you lost, every morsel of your entire being alive to the infinite mystery at all . You had the chalice, the mortar, the tear.'}]

#TEXT TRANSLATION

In [28]:
! pip install sentencepiece

In [29]:
!pip freeze | grep transformers

transformers==4.35.0


In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [28]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")



In [23]:
article_en = transcribe("/content/Jack Sparrow English Dialogue.mp3")

Detected language: en


In [24]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")



In [25]:
model_inputs = tokenizer(article_en, return_tensors="pt")


In [29]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["te_IN"]
)

In [30]:
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


In [31]:
translation

['నేను ఒక కంటి కోసం ఒక బిట్ సెయిల్ కలిగి. యువరాశం ఫౌంటెన్ కనుగొనేందుకు.']